In [1]:
import os
from openmm import Platform, LangevinMiddleIntegrator, XmlSerializer
import openmm.app as app
from openmm.unit import picosecond, kelvin
import parmed
import MDAnalysis as md
from MDAnalysis.analysis import dihedrals
import nglview as ng
import numpy as np
import matplotlib.pyplot as plt
import sys
from pdbfixer import PDBFixer

In [2]:
data_path = '../../data/md_from_scratch/1ytc'
rslt_path = '../../data/md_from_scratch/1ytc/rslt_openmm'
file_name = '1ytc.pdb'
fixed_file_name = '1ytc_fixed.pdb'

In [3]:
# use pdbfixer to fix pdb
# https://htmlpreview.github.io/?https://github.com/openmm/pdbfixer/blob/master/Manual.html
fixer = PDBFixer(filename=os.path.join(data_path, file_name))
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(True)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0)
app.PDBFile.writeFile(fixer.topology, fixer.positions, open(os.path.join(data_path, fixed_file_name), 'w'))

In [4]:
# Input and options
pdbfile = fixed_file_name
psffile = '1ytc_fixed.psf'
pdb = app.PDBFile(os.path.join(data_path, pdbfile))
print(pdb.topology)

# 感觉用 amber14/protein.ff14SB.xml 似乎会更好
# 但实际上看过 amber14-all 就知道它包含了 amber14/protein.ff14SB.xml
# forcefield = app.ForceField('amber14-all.xml')
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

nonbondedMethod = app.NoCutoff
constraints = None
rigidWater = False

dt = 0.002*picosecond
temperature = 300.15*kelvin
friction = 1/picosecond

steps = 50000
logInterval = 100
platform = Platform.getPlatformByName('OpenCL')
# platform = Platform.getPlatformByName('CPU')
# platformProperties = {'Precision': 'mixed'} # use this for CUDA
platformProperties = dict() # use this for CPU

<Topology; 2 chains, 228 residues, 2098 atoms, 2000 bonds>


In [5]:
# Prepare the Simulation
print('Building system...')
topology = pdb.topology
print(topology.getPeriodicBoxVectors())
positions = pdb.positions
system = forcefield.createSystem(topology, nonbondedMethod=nonbondedMethod, constraints=constraints, rigidWater=rigidWater)
print(system.getDefaultPeriodicBoxVectors())
print(system.getForce(0))
print(system.getForce(1))
print(system.getForce(2))
print(dir(system.getForce(0)))
print(system.getForce(0).getBondParameters(0))

# modified by jk
# from openmm import unit
# edge = 1e20 * unit.nanometers
# system.setDefaultPeriodicBoxVectors([edge,0,0], [0,edge,0], [0,0,edge])
# print(system.getDefaultPeriodicBoxVectors())

# save the system
integrator = LangevinMiddleIntegrator(temperature, friction, dt)
simulation = app.Simulation(topology, system, integrator, platform, platformProperties)
# save the topology and the contex
simulation.context.setPositions(positions)
simulation.context.setVelocitiesToTemperature(temperature, 2023)

# save psf file for future visualization
parmed_structure = parmed.openmm.load_topology(topology, system, positions)
parmed_structure.save(os.path.join(data_path, psffile), overwrite=True)

Building system...
(Vec3(x=3.666, y=0.0, z=0.0), Vec3(x=0.0, y=3.666, z=0.0), Vec3(x=0.0, y=0.0, z=13.892)) nm
[Quantity(value=Vec3(x=3.666, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=3.666, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=13.892), unit=nanometer)]
<openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x15b5b4ae0> >
<openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x15b5b4b40> >
<openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x15b5b4b10> >
['__class__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__'

In [6]:
with open(os.path.join(rslt_path, 'state_before_min.xml'), 'w') as f:
    f.write(
        XmlSerializer.serialize(
            simulation.context.getState(getPositions=True,
                                        getVelocities=True,
                                        getForces=True,
                                        getEnergy=True,
                                        getParameters=True,
                                        getParameterDerivatives=True,
                                        getIntegratorParameters=True,
                                        enforcePeriodicBox=True)))

In [7]:
# Minimize
print("\nInitial system energy")
print(simulation.context.getState(getEnergy=True).getPotentialEnergy())
print('Performing energy minimization...')
simulation.minimizeEnergy()
print("\nMinimized system energy")
print(simulation.context.getState(getEnergy=True).getPotentialEnergy())


Initial system energy
59711.796875 kJ/mol
Performing energy minimization...

Minimized system energy
-15200.65234375 kJ/mol


In [8]:
# save file for future use
with open(os.path.join(rslt_path, 'system.xml'), 'w') as f:
    f.write(XmlSerializer.serialize(system))
with open(os.path.join(rslt_path, 'state.xml'), 'w') as f:
    f.write(XmlSerializer.serialize(
            simulation.context.getState(
            getPositions=True, getVelocities=True, getForces=True, 
            getEnergy=True, getParameters=True, getParameterDerivatives=True, 
            getIntegratorParameters=True, enforcePeriodicBox=True)))

In [9]:
# topology from pdbfile, system and state from xml
# use the following code to load simulation to run

# integrator = LangevinMiddleIntegrator(temperature, friction, dt)
# system = XmlSerializer.deserialize(open('system.xml').read())
# simulation = app.Simulation(topology, system, integrator, platform, platformProperties)
# simulation.context.setState(XmlSerializer.deserialize(open('state.xml').read()))

In [10]:
print('Simulating...')
simulation.currentStep = 0
dcdReporter = app.DCDReporter(os.path.join(rslt_path, 'traj-nocutoff.dcd'), logInterval)
dataReporter = app.StateDataReporter(os.path.join(rslt_path, 'traj-nocutoff.log'), logInterval, totalSteps=steps,
    step=True, time=True, speed=True, progress=True, elapsedTime=True, remainingTime=True, potentialEnergy=True, temperature=True, separator='\t')
stdoutReporter = app.StateDataReporter(sys.stdout, logInterval*10, totalSteps=steps,
    step=True, time=True, speed=True, progress=True, elapsedTime=True, remainingTime=True, potentialEnergy=True, temperature=True, separator='\t')
simulation.reporters.append(dataReporter)
simulation.reporters.append(stdoutReporter)
simulation.reporters.append(dcdReporter)
simulation.step(steps)

# Write file with final simulation state
simulation.saveState(os.path.join(rslt_path, 'final_state_nocutoff.xml'))

Simulating...
#"Progress (%)"	"Step"	"Time (ps)"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"	"Elapsed Time (s)"	"Time Remaining"
2.0%	1000	2.0000000000000013	-6595.69091796875	341.4713037072065	0	7.987022399902344e-05	--
4.0%	2000	3.999999999999781	-7944.66357421875	300.53998273519966	339	0.5105118751525879	0:24
6.0%	3000	5.999999999999561	-8643.787109375	299.8286951781408	342	1.0118069648742676	0:23
8.0%	4000	7.999999999999341	-8546.31640625	309.03282836006525	340	1.5226738452911377	0:23
10.0%	5000	10.000000000000009	-8717.66015625	305.4448996482399	340	2.0316836833953857	0:22
12.0%	6000	12.000000000000677	-8704.0673828125	294.8051189068399	340	2.539174795150757	0:22
14.0%	7000	14.000000000001345	-8557.6611328125	307.837637676639	340	3.0496997833251953	0:21
16.0%	8000	16.00000000000201	-8647.37109375	305.97075502385417	340	3.558032751083374	0:21
18.0%	9000	18.000000000000902	-8622.240234375	292.38357282429	340	4.066107749938965	0:20
20.0%	10000	19.999999999999794	-

In [11]:
u = md.Universe(os.path.join(data_path, psffile), os.path.join(rslt_path, 'traj-nocutoff.dcd'))
ng.show_mdanalysis(u, gui=True)

NGLWidget(max_frame=499)